<a href="https://colab.research.google.com/github/comp-aspects-of-appl-linguistics/relation-extraction-utils/blob/master/results/ud_precision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# get stanfordnlp resources 

import os
import stanfordnlp

if not os.path.exists('/root/stanfordnlp_resources'):
  stanfordnlp.download('en', force=True)

In [0]:
# install our utilities

!pip install git+https://github.com/comp-aspects-of-appl-linguistics/relation_extraction_utils.git

In [0]:
# download train.json - whose ID is 1nPpp1zs3_0rkPkCvuz-dSK4YygxO6lw1 (including download of gdrive utility)

!wget -O /tmp/gdrive https://docs.google.com/uc?id=0B3X9GlR6EmbnWksyTEtCM0VfaFE&export=download
!chmod +x /tmp/gdrive
!/tmp/gdrive download 1nPpp1zs3_0rkPkCvuz-dSK4YygxO6lw1

File 'train.json' already exists, use --force to overwrite


In [0]:
# create a sample file (no_relation_sample) with 5% of the sentences and get the ud parses for them captured in file 'no_relation_sample_ucca.csv'

!tac_to_csv --input train.json --relation no_relation | awk 'BEGIN {srand()} !/^$/ { if (FNR==1 || rand() <= .05) print $0}'  > no_relation_sample.csv

In [0]:
!parse_ud --input ./no_relation_sample.csv --output ./no_relation_sample_ud.csv

In [0]:
%%bash

cat > triggers << EOF
create
find
launch
found
s
's
build
her
his
co-founder
start
establish
set
founder
set up
form
EOF

In [0]:
%%bash

cat > paths << "EOF"
!nmod:poss !case >< ^case
^nmod >< ^appos
^obj >< !nsubj
^obj >< ^conj !nsubj
!appos !acl >< !obl
^compound >< ^compound
^obj >< ^acl:relcl ^appos
^obj >< ^acl:relcl
^xcomp >< ^acl
^obj >< ^advcl !nsubj
^nsubj:pass >< !obl
^nmod !appos >< ^appos
^nmod >< !nsubj
!nmod:poss ><
!appos !acl:relcl >< !nsubj
!acl:relcl >< !obl
^obl ^acl:relcl !acl >< !obl
^obj >< ^xcomp !nsubj
^obj >< ^conj ^advcl !nsubj
^obj >< ^acl:relcl !appos
^obj >< ^acl ^obj !nsubj
^nsubj:pass ^acl:relcl >< !acl:relcl !obl
^nsubj:pass !advcl >< !obl
^nsubj:pass >< !obl !nmod !conj !nmod
^nsubj:pass >< !obl !acl !xcomp
^nsubj ^acl:relcl !nmod !nmod:poss ><
^nsubj !obl !nmod !compound >< ^compound
^nsubj !obl >< !nmod
^nsubj !ccomp !ccomp !obl !nmod !appos ><
^nsubj !advcl >< !obl !flat
^nsubj !advcl >< !obl
^nsubj !acl >< !obl
^nsubj >< !obl
^nmod:poss !compound >< ^compound !appos
^nmod:poss >< ^compound
^nmod:poss >< ^appos !appos
^nmod:poss >< ^appos
^nmod ^appos ^nsubj !obj !nmod !compound !punct >< ^punct ^compound ^nmod ^obj !nsubj
^nmod !nmod !acl:relcl >< ^acl:relcl ^nmod ^obl !nsubj
^nmod >< ^acl:relcl
^det !acl:relcl >< !obl
^conj !nmod:poss ><
^conj !conj !nmod:poss ><
^compound ^nmod >< ^conj !appos !nmod
^compound ^nmod >< !nsubj
^compound ^compound ^nmod >< ^appos
^compound ^compound !nmod:poss ><
^compound !nmod:poss !case >< ^case
^appos ^nsubj:pass >< !obl !flat
^appos !nmod:poss !case >< ^case
!nmod:poss !case >< ^case ^nmod:poss
!nmod >< ^nmod ^appos
!appos !acl:relcl >< ^acl:relcl !acl:relcl !nsubj
!appos !acl:relcl >< !obl !nmod
!appos !acl >< !obl !flat
!acl >< !advmod !obl
EOF

In [0]:
!echo "total number of lines:" $(( $(wc -l < no_relation_sample_ud.csv) - 1))
!echo "number of rows with matched triggers (that were not identified) :" $(extract_relations_ud --include_miss --input no_relation_sample_ud.csv paths triggers | grep 'considered the following matching triggers' |sed  's/.*considered the following matching triggers: \(.*\)/\1/' | awk '{ sub("\r$", ""); print }'  | sed '/^$/d' | wc -l )
!echo "number of matches:" $(( $(extract_relations_ud --input no_relation_sample_ud.csv paths triggers | wc -l) - 1 ))
!echo "number of matches filtered by NER:" $(( $(extract_relations_ud --input no_relation_sample_ud.csv paths triggers | append_ner | filter_relations --entity-types ORG PERSON | wc -l) - 1 ))

total number of lines: 2737
number of rows with matched triggers (that were not identified) : 1238
number of matches: 7
number of matches filtered by NER: 0
